# np.roll 활용 data augmentation
 - 참조 : https://dacon.io/competitions/official/235689/codeshare/2347?page=1&dtype=recent&ptype=pub

## np.roll -> shifted data -> new DataFrame

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from tqdm import tqdm_notebook as tn

In [4]:
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test = pd.read_csv('test_features.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [5]:
TL = train_labels[['id','label']]

In [6]:
train = pd.merge(train_features,TL, how = 'left', on = 'id')

In [7]:
train.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,label
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,37
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,37
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,37
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,37
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,37


In [8]:
Label_counts = pd.DataFrame(TL['label'].value_counts())

# <font bord>label 개수 확인 n개보다 적으면 augmentation</font>
 1. lable 개수 확인(<n)
 2. augmentation할 label 저장(list)
 3. label 하나씩 dataframe 불러오기
 4. 불러온 data id 개수 확인
 5. id 한개당 몇개의 shifted data 만들지 계산
     - 한개당 -> int(500/id) - shift(15씩)
 6. shift data -> data frame 변환 id,labe,time등 지정
 7. all_shift에 concat하기

In [9]:
n = 200

In [10]:
#1.
def check_label(data, label, num):
    tem = data[data['label'] == label]
    if len(tem['id'].unique())<num:
        return True
    else : return False

In [2]:
# 데이터 변환
def make_train_data(data):
    train_features = data
    x_train = []

    for uid in tqdm(train_features['id'].unique()):
        temp = np.array(train_features[train_features['id'] == uid].iloc[:,2:], np.float32).T
        x_train.append(temp)

    x_train = np.array(x_train, np.float32)
    x_train = x_train[:,:,:,np.newaxis]
    
    return x_train

In [12]:
#2.
small_label = []
for i in tn(list(train_labels['label'].unique())):
    if check_label(train,i,n):
        small_label.append(i)

c:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
#.데이터 프레임으로 변환
def make_shifted_data(train_data, shift_num,new_id):
    shift_data =  np.roll(train_data, shift_num, axis=2)
    time = list(range(0,600))
    a = pd.DataFrame()
    a['acc_x'] = list(shift_data[0][0].reshape(1,-1)[0])
    a['acc_y'] = list(shift_data[0][1].reshape(1,-1)[0])
    a['acc_z'] = list(shift_data[0][2].reshape(1,-1)[0])
    a['gy_x'] = list(shift_data[0][3].reshape(1,-1)[0])
    a['gy_y'] = list(shift_data[0][4].reshape(1,-1)[0])
    a['gy_z'] = list(shift_data[0][5].reshape(1,-1)[0])
    a['label'] = list(shift_data[0][6].reshape(1,-1)[0])
    a['id'] = new_id
    a['time'] = time
    return a

In [15]:
total_shift_data = pd.DataFrame()

In [16]:
#3 ~ 7.
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
new_id = 1000
for i in tn(small_label):
    print('label, ',i,"진행중")
    tem_data = train[train['label'] == i]
    id_num = list(tem_data['id'].unique())
    how_many_shift = int(n/len(id_num))
    for k in id_num:
        tem_id_data = tem_data[tem_data['id'] == k]
        train_data = make_train_data(tem_id_data)
        for j in range(1,how_many_shift+1):
            shift_num = 19 * j
            shifted_data = make_shifted_data(train_data, shift_num,new_id)
            X_train = shifted_data[features].groupby('id').agg(['max', 'min', 'mean'])
            X_train['label'] = i
            total_shift_data = pd.concat([total_shift_data,X_train])
            print( total_shift_data.shape)
        new_id+=1

c:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


label,  37 진행중
(1, 19)
(2, 19)
(3, 19)
(4, 19)
(5, 19)
(6, 19)
(7, 19)


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

(8, 19)
(9, 19)
(10, 19)
(11, 19)
(12, 19)
(13, 19)
(14, 19)
(15, 19)


...

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.36it/s]

(11384, 19)
(11385, 19)
(11386, 19)
(11387, 19)
(11388, 19)
(11389, 19)
(11390, 19)
(11391, 19)
(11392, 19)



100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 508.15it/s]

(11393, 19)
(11394, 19)
(11395, 19)
(11396, 19)
(11397, 19)
(11398, 19)
(11399, 19)
(11400, 19)


(11401, 19)
(11402, 19)
(11403, 19)
(11404, 19)



In [17]:
total_shift_data.head()

acc_x                        acc_y                         acc_z  \
           max      min      mean       max       min      mean       max   
id                                                                          
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   

                               gy_x                            gy_y            \
           min      mean        max        min      mean        max       min   
id                                                                              
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   

                     gy_z                      label  
          mean        max        min      mean        
id                                                    
1000 -3.359506  55.953827 -79.930031  1.182107    37  
1000 -3.359506  55.953827 -79.930031  1.182107    37  
1000 -3.359506  55.953827 -79.930031  1.182107    37  
1000 -3.359506  55.953827 -79.930031  1.182107    37  
1000 -3.359506  55.953827 -79.930031  1.182107    37

In [18]:
total_shift_data.shape

(11404, 19)